# Running and visualizing self-limitation simulations in three dimensions

In this notebook, we give an example on how to perform three-dimensional lattice Monte-Carlo simulations similar to the ones shown in Fig.4H of [arXiv:2504.13073](https://doi.org/10.48550/arXiv.2504.13073).

## Imports

Note that imports may take a while the first time you run it, as the Blender Python API is a bit heavy

In [1]:
import numpy as np
from json_dump import *
from pathlib import Path
from designs.fcc import (
    CRYSTAL_INTS_CORNER,
    DEFECT_CORNER
)
from contact_utils import ContactMapWrapper
import config as cfg
from plotting import BlenderPlot
from plotting.plot_3d_utils import create_line_material

## Modifiable parameters

### Model parameters
Related to lattice geometry, number of particles, particle-particle interactions 

- `run_name`: string, name to attribute to the run. Input file is saved at `data/{run_name}`, simulation data is saved at `input/{run_name}`
- `n_particles`: number of particles we simulate
- `lattice_size`: number of lattice sites in x, y, and z directions
- `crystal_energy`: $\epsilon_c$ in the article text. Energy of crystalline contacts, in units of the final simulation temperature
- `defect_energy`: $\epsilon_d$ in the main text. Energy of defect contacts, in units of the final simulation temperature. Typically indirectly defined through the `crystal_defect_ratio` parameter, which corresponds to $\epsilon_c / \epsilon_d$
- `repel_energy`: energy of any contact which is not crystalline or defect, which should be strongly repulsive

### Monte-Carlo parameters
Related to annealing profile and number of Monte-Carlo steps

- `n_steps_per_t_per_site`: At each temperature value, `n_steps_t_per_site` * (`lattice_size`)^3 Monte-Carlo steps are performed
- `n_temperature_steps`: Number of temperature steps performed. In this notebook, $\log_{10}(T)$ is decreased from 2 to 0 in constant increments.

### Visualization parameters
- `blender_file`: where to save the 3D Blender file for visualization. By default, will be `3dFigures/{run_name}.blend`
- `view_contacts`: boolean. If set to True, the crystal and defect contacts will be visualized, which can take a long time.
- `cubified`: boolean. If True, the results are plotted inside of a cubic cell instead of an FCC one, which makes for a neater visualization at the cost of introducing artifacts
- `centered`: boolean. If True, the barycenter of the particles is set at the center of simulation box
- `path_to_blender`: string or Path instance, can also be set to None. By default the plotting utilities automatically look for a Blender executable on your machine, but this process may in principle fail on untested systems.
In case it does, please set this variable to the path of your Blender executable.

In [2]:
# ----- PICKING MODEL PARAMETERS -----

# Name of the run
run_name = "notebook_run"

# Lattice dimensions
lattice_size = 12

# Energies
crystal_energy = - 15.0
crystal_defect_ratio = 0.37205221
defect_energy = crystal_energy / crystal_defect_ratio
repel_energy = 100

# Number of particles
n_particles = 400

# ------ PICKING MONTE-CARLO PARAMETERS -----

n_steps_per_t_per_site = n_particles * 24
n_temperature_steps = 120

# ------ PICKING VISUALIZATION PARAMETERS -----
blender_file = f"./3dFigures/{run_name}.blend"
view_contacts = True
cubified = False
centered = True
path_to_blender = None

## Generating model parameters (do not modify)

Model parameters cover the particle and lattice properties, including particle-particle interactions

In [3]:
# ---- GENERATING MODEL PARAMETERS ------
from pathlib import Path

# Define and create paths
data_folder = Path(f"./data/{run_name}")
data_folder.mkdir(exist_ok=True, parents=True)
input_folder = Path(f"./input/{run_name}")
input_folder.mkdir(exist_ok = True, parents = True)
model_path = input_folder / (run_name + "_model_params.json")
mc_path = input_folder / (run_name + "_mc_params.json")

## Dict of model parameters
model_params = {}

In [4]:
# ---------- LATTICE OPTIONS ----------

# Options:
# "chain", "square", "triangular", "cubic", "bcc", "fcc"
model_params["lattice_name"] = "fcc"

# Lattice dimensions
model_params["lx"] = lattice_size
model_params["ly"] = lattice_size  # Has to be 1 for chain
model_params["lz"] = lattice_size  # Has to be 1 for square & triangular

# Number of particle types
# Always set to 1 for the scope of this work
model_params["n_types"] = 1

# Number of particles of each type
model_params["n_particles"] = [n_particles]

In [5]:
# ----- GENERATING ENERGY MATRIX -----
# Separate class to handle contact energies
cmap = ContactMapWrapper.from_lattice_name("fcc", 1, repel_energy)
# Set contacts to the defect and crystal interactions we chose
cmap.set_contacts(CRYSTAL_INTS_CORNER, crystal_energy)
cmap.set_contacts(DEFECT_CORNER, defect_energy)

# And process the couplings into a form processed for the Monte-Carlo simulations
model_params["couplings"] = cmap.get_formatted_couplings()
# Putting the energy scales in the model parameters for convenience. 
# Not actually used for the MC simulations
model_params["crystal_e"] = crystal_energy
model_params["defect_e"] = defect_energy
model_params["repel_e"] = repel_energy

In [6]:
# ----- INITIALIZATION AND RECORDING -----
 
# Initialization option.
# Can be "random" or "from_file" to init from a structure file with the same syntax as the output structure
model_params["initialize_option"] = "random"

# If "initialize_option" is set to "from_file", we must specify the location of
# the input file
# model_params["state_input"] = str(cfg.structures_path/"final_structure.dat")

# Options for average collection
# This should always be set to False
model_params["state_av_option"] = False
# Record energy averages (light)
model_params["e_av_option"] = True
# Record energy after each step (leads to heavy files on long simulations)
model_params["e_record_option"] = True

# Create paths to records if needed
if model_params["e_record_option"]:
    model_params["e_record_output"] = str(data_folder) + "/e_record/"
    Path(model_params["e_record_output"]).resolve().mkdir(
        exist_ok=True, parents=True
    )
if model_params["e_av_option"]:
    model_params["e_av_output"] = str(data_folder) + "/average_e/"
    Path(model_params["e_av_output"]).resolve().mkdir(exist_ok=True, parents=True)

# Pick the probabilities of different moves. Has to sum to 1.
# Options:
""""
        "swap_empty_full",
        "swap_full_full",
        "rotate",
        "mutate",
        "rotate_and_swap_w_empty"
"""

moves_dict = {}
moves_dict["swap_empty_full"] = 1 / 3
moves_dict["rotate"] = 1 / 3
moves_dict["rotate_and_swap_w_empty"] = 1 / 3

model_params["move_probas"] = moves_dict

make_json_file(model_params, model_path)

### Generating Monte-Carlo parameters

Monte-Carlo parameters cover how many temperature steps are ran, how many Monte-Carlo steps are run at each temperature.

In [7]:
mc_params = {}

# Number of MC steps used for equilibration, per site and for each temperature step
mc_params["mcs_eq"] = n_steps_per_t_per_site

# Number of MC steps used for averaging
mc_params["mcs_av"] = 10

# Type of cooling schedule
# Options: "linear", "inverse", "exponential"
# if inverse chosen: specify 1/T as initial and final temperatures
# if exponential chosen: specify log10(T) as initial and final temperatures
mc_params["cooling_schedule"] = "exponential"

# Initial annealing temperature
mc_params["Ti"] = 2

# Final annealing temperature
mc_params["Tf"] = 0

# Number of annealing steps
mc_params["Nt"] = n_temperature_steps

# Option to collect state checkpoints at the end of each temperature cycle
mc_params["checkpoint_option"] = True

# If checkpoint is True, we need to provide the output address for the
# checkpoint files
if mc_params["checkpoint_option"]:
    mc_params["checkpoint_address"] = str(data_folder) + "/structures/"
    Path(mc_params["checkpoint_address"]).resolve().mkdir(
        exist_ok=True, parents=True
    )

# Output location of the final state configuration (must end with "/")
mc_params["final_structure_address"] = mc_params["checkpoint_address"]

make_json_file(mc_params, mc_path)

## Running the simulation

The `run_simulation` command is a thin wrapper around the `./build/app/frusa_mc` executable, which should run cross-platform.

The results of the simulation can be found in `data/{run_name}`, and can consist of up to three folders:

- `average_e/`, containing one file per temperature step, whose contents consist of:
```
[Temperature]
[Average total energy]
[Energy second moment]
```
- `e_record/`, with one file per temperature step. The first line of each file is
the temperature at which samples are recorded. Each following line corresponds to the energy
after each Monte-Carlo step taken at this temperature. Note that, for a large number of
steps, this creates very large files!
- `structures/`, with one file per temperature step. Each file corresponds to the
final lattice configuration at the end of a temperature step. The file consists of two
lines, the first being the type of particle on each lattice site (always 0 here), the other
the orientation of the particle on each site, with orientation -1 corresponding to an empty
site. At the very end of the simulation, a final `final_structure.dat` file is recorded.


In [ ]:
# This is a wrapper to run the simulation itself
import time

t0 = time.time()
cfg.run_simulation(model_path, mc_path, overwrite=True)
t1 = time.time()

print(f"Took {t1-t0}")

At least one of the output folders is already populated!
overwrite flag set to True: running anyway.


## Viewing the simulation results

We display the simulation results using the 3D modeling program Blender, which you will have to install first.

For all the simulation results shown in the article, we used [Blender version 3.4.2](https://download.blender.org/release/Blender4.3/) on MacOS.

Please note that this visualization code has not been ran on Windows or Linux, and that no guarantee is given of it working
on either of these systems.

In [ ]:
# Create Blender wrapper class instance
bp = BlenderPlot.from_model_file(model_file=model_path, blender_bin = path_to_blender)
# Load particle model
bp.load_particle(bp.particle_paths["path_to_one_axis_diagonal_colored_gray"])

if view_contacts:
    bp.plot_canonical_style(
        mc_path,
        blender_file,
        DEFECT_CORNER,
        crystal_contacts=CRYSTAL_INTS_CORNER,
        cubified=cubified,
        centered=centered,
    )
else:
    bp.plot_canonical_style(
        mc_path,
        blender_file,
        DEFECT_CORNER,
        crystal_contacts=CRYSTAL_INTS_CORNER,
        cubified=cubified,
        centered=centered,
    )